In [11]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# ベースURL
base_url = "https://www.jalan.net/ikisaki/map/tokyo/"
response = requests.get(base_url)

# メタタグからエンコーディングを取得して設定
soup = BeautifulSoup(response.content, "html.parser")
meta_tag = soup.find("meta", {"charset": True})

if meta_tag and meta_tag.get("charset"):
    response.encoding = meta_tag.get("charset")
else:
    # 自動推測または固定エンコーディング
    response.encoding = response.apparent_encoding

# 再度パース
soup = BeautifulSoup(response.text, "html.parser")

# 'map-right-inner' を持つ div を取得
map_right_inner_div = soup.find("div", class_="map-right-inner")
if map_right_inner_div:
    a_tags = map_right_inner_div.find_all("a", class_="sub")
    print(f"発見した a タグの数: {len(a_tags)}")
else:
    print("対象の div が見つかりませんでした。")
    a_tags = []

# URLを収集
urls = [urljoin(base_url, a['href']) for a in a_tags if 'href' in a.attrs]

# 特定の文字化けを修正する関数
def fix_encoding(text):
    try:
        return text.encode('latin1').decode('utf-8')
    except (UnicodeEncodeError, UnicodeDecodeError):
        return text  # 修正できない場合そのまま返す

# 各URLにアクセスして詳細情報を取得
for i, url in enumerate(urls, 1):
    print(f"{i}: {url}")
    response = requests.get(url)

    # メタタグでエンコーディングを確認して設定
    detail_soup = BeautifulSoup(response.content, "html.parser")
    meta_tag = detail_soup.find("meta", {"charset": True})
    if meta_tag and meta_tag.get("charset"):
        response.encoding = meta_tag.get("charset")
    else:
        response.encoding = response.apparent_encoding

    # 再度HTMLをパース
    detail_soup = BeautifulSoup(response.text, "html.parser")

    # ターゲットのデータを収集
    div_body = detail_soup.find("div", class_="p-yadoCassette__body p-searchResultItem__body")
    if div_body:
        a_tag = div_body.find("a", class_="jlnpc-yadoCassette__link s16_00 fb")
        h2_tag = div_body.find("h2", class_="p-searchResultItem__facilityName")

        # 詳細ページURL
        detail_url = a_tag['data-href'] if a_tag and 'data-href' in a_tag.attrs else "URLなし"

        # ホテル名
        if h2_tag:
            raw_hotel_name = h2_tag.get_text(strip=True)
            hotel_name = fix_encoding(raw_hotel_name)  # 常に文字化け修正を試みる
        else:
            hotel_name = "ホテル名なし"

        print(f"詳細ページURL: {detail_url}")
        print(f"ホテル名: {hotel_name}")
    else:
        print("対象の情報が見つかりませんでした。")
    print("-" * 50)


発見した a タグの数: 13
1: https://www.jalan.net/130000/LRG_137100/
詳細ページURL: javascript:doYadDetailAd('393714','TGS01419110','115_1_1','2');
ホテル名: 相鉄フレッサイン新橋烏森口（旧ホテルサンルート新橋）
--------------------------------------------------
2: https://www.jalan.net/130000/LRG_138000/
詳細ページURL: javascript:doYadDetailAd('328441','TGS01419146','137_1_1','2');
ホテル名: リッチモンドホテル東京武蔵野
--------------------------------------------------
3: https://www.jalan.net/130000/LRG_136200/
詳細ページURL: javascript:doYadDetailAd('305603','TGS01419074','209_1_1','2');
ホテル名: 傾僑乕儔 搶嫗嬧嵗
--------------------------------------------------
4: https://www.jalan.net/130000/LRG_137400/
詳細ページURL: javascript:doYadDetailAd('343302','TGS01419122','294_1_1','2');
ホテル名: アゴーラプレイス 東京浅草
--------------------------------------------------
5: https://www.jalan.net/130000/LRG_138300/
詳細ページURL: javascript:doYadDetailAd('323781','TGS01419158','63_1_1','2');
ホテル名: the b 嶰尙拑壆乮僓價乕 偝傫偘傫偫傖傗乯
--------------------------------------------------
6: https://www.jalan